In [1]:
import numpy as np
import pandas as pd
import math
from plotnine import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

%matplotlib inline

In [2]:
# bootstrapped computation
def computeLR(target_data, feature_headers, feature_data, bootstraps):

    accuracies = list()
    sensitivities = list()
    specificities = list()

    for _ in range(0,int(bootstraps)):
        
        train_x = ""
        test_x = ""
        train_y = ""
        test_y = ""
    
        diff = 1
    
        while diff > 0.15:
        
            train_x, test_x, train_y, test_y = train_test_split(feature_data, target_data, train_size=0.4, test_size=0.6)
            diff = abs(np.sum(train_y==0) - np.sum(train_y==1)) / len(train_y)
        
        logreg = LogisticRegression(max_iter = 1000)
        
        logreg.fit(train_x, train_y)
        y_pred = logreg.predict(test_x)
        accuracies.append(logreg.score(test_x, test_y))
        
    print(sum(accuracies)/len(accuracies))

In [3]:
def computeLR_write(target_data, feature_headers, feature_data, bootstraps, modelName):

    logreg_models = list()
    
    accuracies = list()
    sensitivities = list() # TPR
    specificities = list() # 
    FPR = list()
    

    for _ in range(0,int(bootstraps)):
        
        train_x = ""
        test_x = ""
        train_y = ""
        test_y = ""
    
        diff = 1
    
        while diff > 0.15:
        
            train_x, test_x, train_y, test_y = train_test_split(feature_data, target_data, train_size=0.4, test_size=0.6)
            diff = abs(np.sum(train_y==0) - np.sum(train_y==1)) / len(train_y)
        
        logreg = LogisticRegression(max_iter = 1000)
        
        logreg.fit(train_x, train_y)
        logreg_models.append(logreg)
        
        y_pred = logreg.predict(test_x)
        accuracies.append(logreg.score(test_x, test_y))
        
        tn, fp, fn, tp = confusion_matrix(test_y, y_pred).ravel() # collapse to 1D array
        sensitivities.append(tp/(tp+fn))
        specificities.append(tn/(tn+fp))
        FPR.append(fp/(fp+tn))
       
    print(sum(accuracies)/len(accuracies))

    
    with open(modelName+"_metrics.txt", "w") as OUT:
        
        OUT.write("endpoint_model" + "\t" + "metric" + "\t" + "acc_sens_spec" + "\n")
        #OUT.write("endpoint_model" + "\t" + "TPR" + "\t" + "FPR" + "\t" + "specificity" + "\t" + "accuracy" + "\n")
        for x in range(0,len(accuracies)):
            #OUT.write(modelName + "\t" + str(sensitivities[x]) + "\t" + str(FPR[x]) + "\t" + str(specificities[x]) + "\t" + str(accuracies[x]) + "\n")
            OUT.write(modelName + "\t" + str(accuracies[x]) + "\t" + "accuracy" + "\n")
            OUT.write(modelName + "\t" + str(sensitivities[x]) + "\t" + "sensitivity" + "\n")
            OUT.write(modelName + "\t" + str(specificities[x]) + "\t" + "specificity" + "\n")
            
    return(logreg_models)

In [6]:
dataset_CR3M = pd.read_csv("dataset/HCC_TACE_metadata_combined_onehot_wTXomics_mod.txt", sep = "\t")
dataset_CR3M = dataset_CR3M.dropna(axis='rows', how='all')

In [7]:
dataset_CR3M['Area_log10'] = np.log10(dataset_CR3M.loc[:,"Area"])

In [8]:
print(dataset_CR3M.shape)
print(list(dataset_CR3M.columns))

(41, 323)
['PatientID', 'BiopsyID_Tumor', 'BiopsyID_NonTumor', 'CR3M', 'Cohort', 'Sex', 'Age', 'Cirrhosis', 'BCLC', 'Child_Pugh', 'NumLiverDiseases', 'Liver.Disease', 'ALD', 'NAFLD', 'HCV', 'HBV', 'HDV', 'NASH', 'AFP_pre', 'AFP_post', 'num_nodules', 'LesionID', 'Segment', 'Edmondson', 'Area', 'Entropy', 'Kurtosis', 'Mpp', 'Skewness', 'Uniformity', 'outliers', 'NT_TU_DE_dist', 'TU_DE_PC1_dist', 'HES2', 'LOC100506022', 'STMN1', 'CLSPN', 'CDCA8', 'CLDN19', 'CDC20', 'KIF2C', 'RAD54L', 'STIL', 'ORC1', 'IL23R', 'DEPDC1', 'PTBP2', 'SASS6', 'RP11-305E17.6', 'GPSM2', 'TRIM45', 'FAM72C', 'FAM72D', 'RNVU1-6', 'LINC00624', 'HIST2H2BE', 'CD1A', 'ITLN2', 'HSPA6', 'NUF2', 'ANKRD45', 'ASPM', 'KIF14', 'UBE2T', 'FAM72A', 'NEK2', 'DTL', 'CENPF', 'HHIPL2', 'EXO1', 'LINC00200', 'MCM10', 'ACBD7', 'LOC101929475', 'ZWINT', 'CDK1', 'DYDC2', 'KIF11', 'HELLS', 'LOXL4', 'CALHM2', 'ADAM12', 'MKI67', 'RRM1', 'USH1C', 'OTOG', 'SAA4', 'SAA2', 'SAA1', 'E2F8', 'KIF18A', 'CD44', 'SLC35C1', 'FAM111B', 'CDCA5', 'POLD3', '

In [9]:
dataset_CR3M["CR3M"].value_counts()

0    24
1    17
Name: CR3M, dtype: int64

In [10]:
discovery_dataset = dataset_CR3M[~dataset_CR3M['BiopsyID_Tumor'].isin(["D212","D601","D999","E001","E096","E185","E209","D180"])]

In [11]:
validation_dataset = dataset_CR3M[dataset_CR3M['BiopsyID_Tumor'].isin(["D212","D601","D999","E001","E096","E185","E209","D180"])]

In [12]:
target_data = np.array(discovery_dataset.loc[:,'CR3M']).ravel() # ravel converts to 1D array
feature_headers = ['Area_log10']
feature_data = np.array(preprocessing.scale(discovery_dataset.loc[:,feature_headers]))

computeLR(target_data, feature_headers, feature_data, 500)

0.7920000000000019


In [13]:
target_data = np.array(discovery_dataset.loc[:,'CR3M']).ravel() # ravel converts to 1D array
feature_headers = ['FAM111B','HPRT1']
feature_data = np.array(preprocessing.scale(discovery_dataset.loc[:,feature_headers]))

computeLR(target_data, feature_headers, feature_data, 500)

0.8528999999999994


In [14]:
target_data = np.array(discovery_dataset.loc[:,'CR3M']).ravel() # ravel converts to 1D array
feature_headers = ['Area_log10','FAM111B','HPRT1']
feature_data = np.array(preprocessing.scale(discovery_dataset.loc[:,feature_headers]))

computeLR(target_data, feature_headers, feature_data, 500)

0.8977999999999972


In [17]:
target_data = np.array(discovery_dataset.loc[:,'CR3M']).ravel() # ravel converts to 1D array
feature_headers = ['Area_log10','FAM111B','HPRT1']
feature_data = np.array(preprocessing.scale(discovery_dataset.loc[:,feature_headers]))

train_x = ""
test_x = ""
train_y = ""
test_y = ""

diff = 1
    
while diff > 0.15:
        
    train_x, test_x, train_y, test_y = train_test_split(feature_data, target_data, train_size=0.4, test_size=0.6)
    diff = abs(np.sum(train_y==0) - np.sum(train_y==1)) / len(train_y)
        
logreg = LogisticRegression(max_iter = 1000)
        
logreg.fit(train_x, train_y)
y_pred = logreg.predict(test_x)

print(test_y)
print(y_pred)

[1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0]
[1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 0]


In [18]:
# this can also be bootstrapped: E001 is the only sample which is consistently misclassified

validation_target = np.array(validation_dataset.loc[:,'CR3M']).ravel() # ravel converts to 1D array
validation_features = ['Area_log10','FAM111B','HPRT1']
validation_data = np.array(preprocessing.scale(validation_dataset.loc[:,validation_features]))

validation_pred = logreg.predict(validation_data)

#print(np.array(validation_dataset.loc[:,'TumorID.x']).ravel())
print(np.array(validation_dataset.loc[:,'BiopsyID_Tumor']).ravel())
print(validation_target)
print(validation_pred)

['D180' 'E209' 'D999' 'E001' 'D212' 'D601' 'E185' 'E096']
[0 0 0 0 1 1 1 1]
[0 0 0 1 1 1 1 0]


In [ ]:
# repeat the above cell for new data, provided it is preprocessed along with the discovery cohort samples as described in the manuscript